In [2]:
import os, json
import pandas as pd

In [38]:
data = []

path_to_json = './data/'

for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
    with open(path_to_json + file_name, encoding="utf8") as json_file:
        file = json.load(json_file)
        data.extend(file)

In [40]:
file[0]['data']

[[1681084800000,
  'First Solar, Inc. to Announce First Quarter 2023 Financial Results on April 27, 2023',
  3.3518989321],
 [1680134400000,
  'EDP Renewables Places Multi-Year Order for 1.8 GW of First Solar Modules',
  -0.5195424749],
 [1680048000000,
  'Indian Government Awards Production Linked Incentives to First Solar’s India Manufacturing Facility',
  0.5437856845],
 [1677542400000,
  'First Solar, Inc. Announces Fourth Quarter and Full Year 2022 Financial Results and 2023 Guidance',
  15.0449653948],
 [1677542400000,
  'Lightsource bp Places Multi-Year Order for 4 GW of First Solar Modules',
  15.0449653948],
 [1676332800000,
  'First Solar, Inc. to Announce Fourth Quarter and Full Year 2022 Financial Results and 2023 Financial Guidance on February 28, 2023',
  6.0222817626],
 [1673481600000,
  'First Solar Completes Sale of 141 MW Luz del Norte Power Plant to Toesca',
  3.9347334712],
 [1671062400000,
  'National Grid Renewables Orders 1.6 GW of First Solar Modules',
  -2.2053

In [9]:
json.parse(data[0])

AttributeError: module 'json' has no attribute 'parse'